In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/media/paulati/Nuevo vol/paula/dh/2021/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Limpieza de datos. Apply. Expresiones regulares.

La limpieza es un paso necesario en todo proyecto de datos. 

Podemos resumir el proceso de limpieza de datos en las siguientes cinco tareas:

**1. Resolver problemas de formato y asignar tipos de datos correctos.**

Por ejemplo, cuando al pasar de CSV a Pandas una fecha no se importa correctamente como puede ser el caso de un campo fecha donde se importa 20090609231247 en lugar de 2009-06-09 23:12:47.

El formato en que se encuentran los datos determina qué operaciones pueden realizarse sobre ellos.

**2. Estandarizar categorías.**

Cuando los datos se recolectaron con un sisstema que no tiene valores tipificadaos, valores que representan la misma categoría pueden estar expresados de forma distinta. Por ejemplo: Arg, AR, Argentina

**3. Corregir valores erróneos.**

Por ejemplo: un valor numérico o inválido para describir el género; o una edad representada por un número negativo o mucho mayor que 100.

**4. Completar datos faltantes.**

Los datasets del mundo real suelen venir con datos faltantes que responden a información que se perdió o nunca se recolectó. Existen varias técnicas para completar datos faltantes. Al proceso de completar datos faltantes se lo llama "imputación".

**5. Organizar el dataset.**

Es importante estructurar las filas y columnas de la forma más conveniente. Para hacerlo se pueden aplicar las reglas del "tidy data".



## Dataset

El Met (Metropolitan Museum of Art) provee datasets de información de más de 420.000 piezas de arte en su colección.

https://github.com/metmuseum/openaccess/

Los problemas que presentan este dataset incluyen:
* Valores faltantes
* Posibles duplicaciones
* Campos con mezcla de campos de tipo numérico con cadenas de caracteres

En esta clase vamos a detectar algunos de los qué campos presentan problemas, y solucionarlos usando las herramientas que adquirimos en las prácticas guiadas.


## Ejercicio 1:

Vamos a leer en la variable `data` los datos del archivo /M2/CLASE_05_Limpieza_de_datos/Data/MetObjects_sample.csv en un `DataFrame` de pandas con el método `read_csv` 

Veamos cuántas filas y columnas tiene el DataFrame data

In [2]:
## [BORRAR_PRESENCIAL]

import pandas as pd

# local
data_location = "../Data/MetObjects_sample.csv"
# colab
# data_location = ""

data = pd.read_csv(data_location, low_memory=False)

print("cantidad de filas: " + str(data.shape[0]))
print("cantidad de columnas: " + str(data.shape[1]))


cantidad de filas: 4743
cantidad de columnas: 52


## Ejercicio 2: Formato y tipos de datos

Vamos a ver de qué tipo de datos es cada columna del DataFrame, y vamos a convertir o dar formato **a alguna de las columnas** que tienen tipo de datos incorrecto.

### 2.a Detectar las columnas que tienen tipo de datos incorrecto

In [3]:
## [BORRAR_PRESENCIAL]
data.dtypes

Unnamed: 0                   int64
Object Number               object
Is Highlight                  bool
Is Public Domain              bool
Is Timeline Work              bool
Object ID                    int64
Department                  object
AccessionYear               object
Object Name                 object
Title                       object
Culture                     object
Period                      object
Dynasty                     object
Reign                       object
Portfolio                   object
Artist Role                 object
Artist Prefix               object
Artist Display Name         object
Artist Display Bio          object
Artist Suffix               object
Artist Alpha Sort           object
Artist Nationality          object
Artist Begin Date           object
Artist End Date             object
Artist Gender               object
Artist ULAN URL             object
Artist Wikidata URL         object
Object Date                 object
Object Begin Date   

### 2.b AccessionYear

Analizar la columna AccessionYear que fue leída como object, y debería ser int.

¿Qué valores toma ese campo? ¿Cómo se distribuyen esos valores? ¿Hay valores nulos?

Queremos extraer el dato año de los valores no numéricos, y crear una nueva columna en el DataFrame de tipo int que se llame AccessionYearClean y tenga estos valores.

Para eso vamos a usar expresiones regulares, apply y lambda.

Observación: si la columna tiene valores NaN no vamos a poder convertirla al tipo int. Una opción es reemplazar los valores NaN por algún entero antes de convertir (`fillna`). La otra opción es dejar la columna como tipo float (el tipo de NaN es float).


In [4]:
## [BORRAR_PRESENCIAL]
#Con value_counts() vemos que algunos de los registros tienen una fecha en lugar de tener sólo el año.
#Contamos los nulll

data.AccessionYear.value_counts()

1963          220
1917          140
1874          126
1929           99
2011           98
             ... 
1986-12-29      1
2020-03-23      1
1873            1
2005-02-15      1
1871            1
Name: AccessionYear, Length: 142, dtype: int64

In [5]:
## [BORRAR_PRESENCIAL]
data.AccessionYear.isnull().sum()

1000

In [6]:
## [BORRAR_PRESENCIAL]
import re
import numpy as np 

pattern_fecha = "(?P<year>\d\d\d\d)(?P<month_day>\-\d\d\-\d\d)*"
pattern_fecha_regex =  re.compile(pattern_fecha)
#resultado_fechas = data.AccessionYear.apply(lambda x: x if (x is np.NaN) | (x is None) else pattern_fecha_regex.match(x))
resultado_fechas = data.AccessionYear.apply(lambda x: x if x is np.NaN else pattern_fecha_regex.match(x))

year_match = resultado_fechas.apply(lambda x: x if x is np.NaN else x.group("year"))
print(year_match.dtype)


#opcion 1
year_match_fill = year_match.fillna(0)
year_match_fill_numeric = year_match_fill.astype(int)

# opcion 2
year_match_numeric = year_match.astype(float)

#data["AccessionYearClean"] = year_match_numeric
data["AccessionYearClean"] = year_match_fill_numeric
data.dtypes

object


Unnamed: 0                   int64
Object Number               object
Is Highlight                  bool
Is Public Domain              bool
Is Timeline Work              bool
Object ID                    int64
Department                  object
AccessionYear               object
Object Name                 object
Title                       object
Culture                     object
Period                      object
Dynasty                     object
Reign                       object
Portfolio                   object
Artist Role                 object
Artist Prefix               object
Artist Display Name         object
Artist Display Bio          object
Artist Suffix               object
Artist Alpha Sort           object
Artist Nationality          object
Artist Begin Date           object
Artist End Date             object
Artist Gender               object
Artist ULAN URL             object
Artist Wikidata URL         object
Object Date                 object
Object Begin Date   

## Ejercicio 3: Categorias - Valores erróneos

Miremos ahora el campo "Artist Gender"

¿Qué valores toma ese campo? ¿Cómo se distribuyen esos valores? ¿Hay valores nulos?

Queremos definir como categorías válidas Male, Female y Unknown

Y crear una nueva columna en el DataFrame que se llame ArtistGenderClean y tenga estos valores.

Para eso vamos a usar expresiones regulares, apply y lambda.

¿Podemos deducir cómo está representada la categoría Male en el dataset original?

Nota: La propuesta que hacemos para limpiar este campo no es del todo correcta, y vamos a ver por qué más adelante. Pero sirve como ejercicio.


In [7]:
## [BORRAR_PRESENCIAL]

data["Artist Gender"].value_counts()                     

|                           583
||                          170
Female                       81
Female|                      43
|||                          41
|Female                      23
||||                         20
||||||                        7
|||||||                       3
Female||                      3
|||||                         3
|Female|                      2
|||||||||||                   2
Female||Female                2
Female|Female                 1
|||||||||||||||||             1
|Female|Female|||Female       1
|Female||                     1
|||||||||||||||Female|||      1
||||||||                      1
||Female                      1
||Female|                     1
|||Female|||||                1
Name: Artist Gender, dtype: int64

In [8]:
## [BORRAR_PRESENCIAL]

data["Artist Gender"].isnull().sum()

3751

In [9]:
## [BORRAR_PRESENCIAL]
#Comenzamos asignando unknown a todos los nulos
data["ArtistGenderClean"] = data["Artist Gender"].fillna('Unknown')
data["ArtistGenderClean"].value_counts()

Unknown                     3751
|                            583
||                           170
Female                        81
Female|                       43
|||                           41
|Female                       23
||||                          20
||||||                         7
|||||||                        3
Female||                       3
|||||                          3
|Female|                       2
|||||||||||                    2
Female||Female                 2
Female|Female                  1
|||||||||||||||||              1
|Female|Female|||Female        1
|Female||                      1
|||||||||||||||Female|||       1
||||||||                       1
||Female                       1
||Female|                      1
|||Female|||||                 1
Name: ArtistGenderClean, dtype: int64

In [10]:
## [BORRAR_PRESENCIAL]
# Vamos a reemplazar las sucesiones de pipe por un string vacío
pattern_pipes = "\|+"
pattern_pipes_regex = re.compile(pattern_pipes)
cadena_reemplazo = ""
data["ArtistGenderClean"] = data["ArtistGenderClean"].apply(lambda x: pattern_pipes_regex.sub(cadena_reemplazo, x))
data["ArtistGenderClean"].value_counts()


Unknown               3751
                       831
Female                 157
FemaleFemale             3
FemaleFemaleFemale       1
Name: ArtistGenderClean, dtype: int64

In [11]:
## [BORRAR_PRESENCIAL]
# Vamos a reemplazar las sucesiones de Female por un Female
pattern_female = "(Female)+"
pattern_female_regex = re.compile(pattern_female)
cadena_reemplazo = "Female"
data["ArtistGenderClean"] = data["ArtistGenderClean"].apply(lambda x: pattern_female_regex.sub(cadena_reemplazo, x))
data["ArtistGenderClean"].value_counts()

Unknown    3751
            831
Female      161
Name: ArtistGenderClean, dtype: int64

In [12]:
## [BORRAR_PRESENCIAL]
# Vamos a reemplazar ahora "" por Male usando una máscara booleana
empty_mask = data["ArtistGenderClean"] == ""
data.loc[empty_mask, "ArtistGenderClean"] = "Male"
data["ArtistGenderClean"].value_counts()

Unknown    3751
Male        831
Female      161
Name: ArtistGenderClean, dtype: int64

## Ejercicio 4: Imputación

Vamos a analizar ahora los campos "Object Date", "Object Begin Date", "Object End Date"

**4.a ¿Cuántos valores nulos hay en "Object Date"? ¿Cuántos en "Object Begin Date"? ¿Cuántos en "Object End Date"?**


In [13]:
## [BORRAR_PRESENCIAL]
data["Object Date"].isnull().value_counts()

False    3173
True     1570
Name: Object Date, dtype: int64

In [14]:
## [BORRAR_PRESENCIAL]
data["Object Begin Date"].isnull().value_counts()

False    4743
Name: Object Begin Date, dtype: int64

In [15]:
## [BORRAR_PRESENCIAL]
data["Object End Date"].isnull().value_counts()

False    4743
Name: Object End Date, dtype: int64

**4.b Usaremos los valores de "Object Begin Date" o "Object End Date" para imputar los valores de "Object Date" con alguno de esos dos campos.**

1) Vamos a crear una columna nueva ("Object Date 4b") donde copiamos todos los datos de Object Date (para no cambiar los valores originales y nos sirvan para el próximo ejercicio)

2) Vamos a rellenar la columna "Object Date 4b" con la estrategia que planteamos.

In [16]:
## [BORRAR_PRESENCIAL]

data["Object Date 4b"] = data["Object Date"]

# para crear la máscara pueda usar cualquiera de las dos columnas, la original o la copia
mask_object_date_null = data["Object Date"].isnull()

mask_object_date_null.sum()

# ahora modifico los valores de la columna "Object Date 4b"
data.loc[mask_object_date_null, "Object Date 4b"] = data.loc[mask_object_date_null, "Object End Date"]

# veo cómo se completaron los valores de "Object Date 4b"
data.loc[mask_object_date_null, ["Object Date 4b", "Object Begin Date", "Object End Date"]]

,Object Date 4b,Object Begin Date,Object End Date
51,0,0,0
103,330,-7000,330
122,0,0,0
136,0,0,0
159,0,0,0
...,...,...,...
2498,1450,1440,1450
2499,-30,-7000,-30
2500,1911,1644,1911
2501,1536,1526,1536


**4.c Usaremos aleatoriamente los valores de "Object Begin Date" o "Object End Date" para imputar los valores de "Object Date" con alguno de esos dos campos.**

Para eso definimos una función get_fill_value que recibe como parámetro una fila da data, y si el valor del campo "Object Date" es nulo devuelve aleatoriamente el valor del campo "Object Begin Date" u "Object End Date" de ese registro.

(En este ejercicio practicamos todo!)

In [17]:
## [BORRAR_PRESENCIAL]

def get_fill_value(row):    
    if row["Object Date"] is np.NaN:
        random_generator = np.random.default_rng()
        rnd = random_generator.uniform()
        if rnd < 0.5:
            result = row["Object Begin Date"]
        else:
            result = row["Object End Date"]
    else:
        result = row["Object Date"]
    return result    

data["Object Date Fill"] = data.apply(get_fill_value, axis = 1)

#defino una máscara que me muestre los registros que completé:
mask_fill = data["Object Date"] != data["Object Date Fill"]
# cantidad de registros que completé:
print(mask_fill.sum())
# miro qué hizo en los que completé:
print(data.loc[mask_fill, ["Object Date", "Object Date Fill", "Object Begin Date", "Object End Date"]])


1570
     Object Date Object Date Fill  Object Begin Date  Object End Date
51           NaN                0                  0                0
103          NaN            -7000              -7000              330
122          NaN                0                  0                0
136          NaN                0                  0                0
159          NaN                0                  0                0
...          ...              ...                ...              ...
2498         NaN             1450               1440             1450
2499         NaN              -30              -7000              -30
2500         NaN             1911               1644             1911
2501         NaN             1526               1526             1536
2502         NaN              907                618              907

[1570 rows x 4 columns]


Vamos a contar ahora cuántos registros rellenó con los valores de Object Begin Date y cuántos con Object End Date

In [18]:
## [BORRAR_PRESENCIAL]

# máscara que define qué registros fueron completados:
mask_fill = data["Object Date"] != data["Object Date Fill"]

# máscara que define qué registros fueron completados, y usamos Object Begin Date:
object_begin_date_fill_mask = np.logical_and(mask_fill, data["Object Date Fill"] == data["Object Begin Date"])

# máscara que define qué registros fueron completados, y usamos Object End Date:
object_end_date_fill_mask = np.logical_and(mask_fill, data["Object Date Fill"] == data["Object End Date"])

print(object_begin_date_fill_mask.sum())

print(object_end_date_fill_mask.sum())

# bien! rellenó 50% con cada columna. 
# ojo con esto que si Object Begin Date es igual a Object End Date los cuento dos veces! (por eso suman más que 15002)

858
869


## Nota: Organizar el dataset

Para llevar a cabo esta tarea necesitamos algunas herramientas que veremos en la segunda parte de Pandas.

En esa clase vamos a volver a este ejercicio y resolver este punto.

Los campos que vamos a analizar son "Artist Nationality" y "Artist Display Name"

In [19]:
data["Artist Nationality"].value_counts()

American                                                                                        329
Austrian                                                                                        153
French                                                                                          146
Japanese                                                                                         96
Italian                                                                                          93
                                                                                               ... 
German|Irish|Belgian                                                                              1
Italian|Italian|Italian|Italian|Italian|Italian|Italian|Italian|Italian                           1
Spanish|Spanish|French                                                                            1
British|Indian|Indian                                                                             1


In [20]:
data["Artist Display Name"].value_counts() 

Walker Evans                                    35
Unknown                                         32
W. Duke, Sons & Co.                             21
Kinney Brothers Tobacco Company                 20
Goodwin & Company                               19
                                                ..
Herman Herzog                                    1
Roger Vivier|Marc Bohan|House of Dior            1
Diego Rivera                                     1
George Bellows                                   1
American Art Union|James Smillie|Thomas Cole     1
Name: Artist Display Name, Length: 1595, dtype: int64